In [39]:
import torch
import sys
sys.path.append('/home/zjy/project/MetaIM')
pwd = '/home/zjy/project/MetaIM/data'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [40]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root=pwd+'/cora', name='cora')
data = cora_dataset[0]
edge_index = data.edge_index

In [41]:
import numpy as np
individual_infection_path = pwd+'/for_meta/cora_individual_infection_sir_200_sample_500.npy'
seeds_infection_path = pwd+'/for_meta/cora_seed_infection_sir_200_sample_500.npy'

individual_infection = np.load(individual_infection_path)
seeds_infection = np.load(seeds_infection_path)
individual_infection.shape,seeds_infection.shape

((2708, 2708), (1000, 2, 2708))

In [42]:
import torch
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sp

# 转换为 scipy 稀疏矩阵
adj = to_scipy_sparse_matrix(edge_index)

adj = torch.Tensor(adj.toarray()).to_sparse()
adj = adj.to(device)


In [43]:
def edge_index_to_adj(edge_index, num_nodes):
    adj = torch.zeros((num_nodes, num_nodes))
    for i in range(edge_index.size(1)):
        start_node = edge_index[0, i]
        end_node = edge_index[1, i]
        adj[start_node, end_node] = 1
        adj[end_node, start_node] = 1  # 如果是有向图，可以不需要这行
    return adj

adj_n = edge_index_to_adj(edge_index, individual_infection.shape[0])

In [44]:


def graph_laplacian_embedding(adjacency_matrix, num_dimensions):
    # 将邻接矩阵转换为PyTorch张量
    adjacency_matrix = torch.tensor(adjacency_matrix, dtype=torch.float32)
    
    # Step 1: 计算度矩阵
    degree_matrix = torch.diag(adjacency_matrix.sum(dim=1))
    
    # Step 2: 计算拉普拉斯矩阵
    laplacian_matrix = degree_matrix - adjacency_matrix
    
    # Step 3: 计算拉普拉斯矩阵的特征向量和特征值
    eigenvalues, eigenvectors = torch.linalg.eigh(laplacian_matrix)
    
    # Step 4: 选择低维表示
    # 取特征值非零的前 num_dimensions 个特征向量作为嵌入结果
    embedding = eigenvectors[:, :num_dimensions]
    
    return embedding



In [45]:
laplace_emb = graph_laplacian_embedding(adj_n, 64)
laplace_emb = laplace_emb.to(device)

/tmp/ipykernel_2522300/1279567966.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adjacency_matrix = torch.tensor(adjacency_matrix, dtype=torch.float32)


In [46]:
seed_num = int(seeds_infection[0][0].sum())

In [47]:
from torch.utils.data import Dataset, DataLoader, random_split


class CustomDataset(Dataset):
    def __init__(self, individual_infection,seeds_infection, seed_num):
        self.individual_infection = individual_infection
        self.seeds_infection = seeds_infection
        self.feat_shape = (len(individual_infection), seed_num)

    def __len__(self):
        return len(self.seeds_infection)

    def __getitem__(self, idx):
        seeds= np.nonzero(self.seeds_infection[idx][0])[0]
        
        feature = torch.zeros(self.feat_shape[0],self.feat_shape[1])
        for i in range(len(seeds)):
            seed_i_infection = torch.tensor(self.individual_infection[seeds[i]])
            feature[:, i] = seed_i_infection
            
        label = self.seeds_infection[idx][1]
        
        return self.seeds_infection[idx][0], feature, label

dataset = CustomDataset(individual_infection, seeds_infection, seed_num)

In [48]:
# 定义划分比例
train_ratio = 0.8
test_ratio = 0.2

# 划分数据集
train_dataset, test_dataset = random_split(dataset, [int(len(dataset)*train_ratio), int(len(dataset)*test_ratio)])

train_batch_size = 32
test_batch_size = 2

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

In [49]:
import torch
import torch.nn as nn

from torch_geometric.nn import GATConv
from torch.optim import Adam, SGD
import torch.nn.functional as F

class GAT(nn.Module):
    def __init__(self, feat_dim, latent_dim, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        self.linear1 = nn.Linear(latent_dim, 128)
        self.conv1 = GATConv(feat_dim, hidden_channels, heads=num_heads)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels * num_heads, heads=1)
        self.linear2 = nn.Linear(hidden_channels * num_heads+128, 1)

    def forward(self, feat_i, x_i, edge_index):
        x = self.conv1(feat_i, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        
        latten = F.relu(self.linear1(x_i))
        x =  torch.cat((x, latten), dim=-1)
        x = self.linear2(x)
        return F.relu(x)

In [50]:
feat_num = seed_num
latent_dim = 64

forward_model = GAT(feat_num,latent_dim, 512, 1, 4)

optimizer = Adam([{'params': forward_model.parameters()}], 
                 lr=0.001)

adj = adj.to(device)
forward_model = forward_model.to(device)
forward_model.train()

GAT(
  (linear1): Linear(in_features=64, out_features=128, bias=True)
  (conv1): GATConv(135, 512, heads=4)
  (conv2): GATConv(2048, 2048, heads=1)
  (linear2): Linear(in_features=2176, out_features=1, bias=True)
)

In [51]:
import torch

# 假设 A 和 emb 是随机生成的示例张量
A = torch.randint(0, 2, (2700,)).double()
emb = torch.randn(2700, 64).double()

# 将emb进行转置
emb_transpose = emb

# 计算A与emb的转置的乘积
result = torch.matmul(A, emb_transpose)

# 打印结果
print("Result:")
print(result)


Result:
tensor([-41.1103,  13.6770,  57.3309,  13.6348,   0.7261, -34.0031, -65.9632,
        -16.3272,  20.3972, -38.0744, -39.8438, -55.5805, -19.1470,  43.8956,
         47.9416,  -7.9666, -36.6753, -36.7269,  27.8084,   5.2519, -29.6941,
        -40.0241, -12.2777,  45.2819,  14.2243, -23.7032,  30.9957, -19.4507,
        -35.1556,  39.8545,   1.3376, -21.7191,  13.2834, -19.3445, -47.2657,
        -18.1323,  13.0394, -10.4747, -29.0267, -16.0358, -11.0079,  39.8639,
         26.4519, -22.0124,  29.3506, 115.8004,  52.6481, -15.6346,  -6.8232,
        -13.7503,   2.6397, -24.4359, -28.6654,   6.1546, -19.2340,  54.5242,
          2.2569, -10.7268, -31.3181,  31.2496, -35.7503,  -3.7822, -65.6443,
         37.7439], dtype=torch.float64)


In [52]:
edge_index = edge_index.to(device)
top_num = 500
# encoder = vae_model.Encoder

for epoch in range(2000):

    total_overall = 0
    forward_loss = 0

    mean_train_accuracy = 0
    for batch_idx, x_feature_label in enumerate(train_loader):  
        x =  x_feature_label[0].to(device)     
        features = x_feature_label[1].to(device)
        labels = x_feature_label[2].to(device)
        optimizer.zero_grad()
        
        loss = 0
        train_accuracy = 0
        for i, feat_i in enumerate(features):
            
            # log_sum = torch.sum(torch.log(1 - x_i), dim=1, keepdim=True)
            # feat = 1 - torch.exp(log_sum)
            x_i = x[i]
            x_i = torch.matmul(x_i, laplace_emb)
            
            
            # x_i = encoder(x_i).detach()
            # x_i = torch.rand(latent_dim).to(device)
            # x_i = normalize_features(x_i)
            x_i = x_i.expand(features.shape[1], -1)
            # final_feat_i =  torch.cat((feat_i, x_i), dim=1)
            
            
            y_i = labels[i]
            y_hat = forward_model(feat_i, x_i, edge_index)
            _, top_indices_true = torch.topk(y_i.clone(), top_num)
            label_2 = torch.zeros(y_i.shape).to(device)
            label_2[top_indices_true] = 1
            
            _, top_indices_predict = torch.topk(y_hat.clone().squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            accuracy_i = len(intersection) / top_num       
            train_accuracy += accuracy_i 

            forward_loss = 0.5*F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum') + F.mse_loss(y_hat.squeeze(-1), label_2, reduction='sum')    
            loss += forward_loss    
        
        total_overall += loss.item()    
        train_accuracy /= len(features)
        mean_train_accuracy = train_accuracy
        loss = loss/features.size(0)
        loss.backward()
        optimizer.step()
        # for p in forward_model.parameters():
        #     p.data.clamp_(min=0)
        
        
    print("Epoch: {}".format(epoch+1), 
        "\tTotal: {:.4f}".format(total_overall / train_batch_size),
        "\tMean_train_accuracy: {:.4f}".format(mean_train_accuracy),
        )  
    
    mean_accuracy = 0
    mean_accuracy_sum = 0

    
    for batch_idx, x_feature_label in enumerate(test_loader):  
        x =  x_feature_label[0].to(device)
        features = x_feature_label[1].to(device)
        labels = x_feature_label[2].to(device)
        
        accuracy = 0
        accuracy_sum = 0
        
        for i, feat_i in enumerate(features):
            # x_i = x[i]
            # x_i = encoder(x_i).detach()
            # x_i = normalize_features(x_i)
            x_i = x[i]
            x_i = torch.matmul(x_i, laplace_emb)
            x_i = x_i.expand(features.shape[1], -1)
            # final_feat_i =  torch.cat((feat_i, x_i), dim=1)
            y_i = labels[i]
            _, top_indices_true = torch.topk(y_i, top_num)
            
            y_hat = forward_model(feat_i, x_i, edge_index)
            
            _, top_indices_predict = torch.topk(y_hat.squeeze(-1), top_num)
            
            sum_pre = torch.sum(feat_i, dim=1, keepdim=True)
            _, top_indices_sum = torch.topk(sum_pre.squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()
            
            list_sum = top_indices_sum.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            
            intersection_sum = list(set(list1) & set(list_sum))
            
            accuracy_i = len(intersection) / top_num       
            accuracy += accuracy_i 
            accuracy_sum += len(intersection_sum) / top_num  
        accuracy /= test_batch_size
        accuracy_sum/= test_batch_size
        mean_accuracy = accuracy
        mean_accuracy_sum = accuracy_sum
        break
    
    print(
        "\tMean_test_accuracy: {:.4f}".format(mean_accuracy),
        "\tMean_test_accuracy_sum: {:.4f}".format(mean_accuracy_sum)
        )  

    

Epoch: 1 	Total: 10888.2294 	Mean_train_accuracy: 0.5281
	Mean_test_accuracy: 0.5780 	Mean_test_accuracy_sum: 0.5560
Epoch: 2 	Total: 8698.3886 	Mean_train_accuracy: 0.5928
	Mean_test_accuracy: 0.6320 	Mean_test_accuracy_sum: 0.5560
Epoch: 3 	Total: 8324.8123 	Mean_train_accuracy: 0.6232
	Mean_test_accuracy: 0.6440 	Mean_test_accuracy_sum: 0.5560
Epoch: 4 	Total: 8071.5876 	Mean_train_accuracy: 0.6322
	Mean_test_accuracy: 0.6420 	Mean_test_accuracy_sum: 0.5560
Epoch: 5 	Total: 8044.2961 	Mean_train_accuracy: 0.6346
	Mean_test_accuracy: 0.6580 	Mean_test_accuracy_sum: 0.5560
Epoch: 6 	Total: 7766.9346 	Mean_train_accuracy: 0.6466
	Mean_test_accuracy: 0.6760 	Mean_test_accuracy_sum: 0.5560
Epoch: 7 	Total: 7569.1193 	Mean_train_accuracy: 0.6496
	Mean_test_accuracy: 0.6740 	Mean_test_accuracy_sum: 0.5560
Epoch: 8 	Total: 7457.2259 	Mean_train_accuracy: 0.6609
	Mean_test_accuracy: 0.6800 	Mean_test_accuracy_sum: 0.5560
Epoch: 9 	Total: 7299.6323 	Mean_train_accuracy: 0.6573
	Mean_test_accu